In [2]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
from pyspark.mllib.util import MLUtils
import pandas as pd
from pyspark.mllib.linalg import SparseVector
from pyspark.mllib.regression import LabeledPoint

In [3]:
sc = SparkContext('local[*]')
spark = SparkSession(sc)

In [4]:
data = spark.read.format("csv").load("final_ball_0.csv")
data = data.rdd
#finalData = data.map(lambda row: [float(row[0]),float(row[1]),float(row[2]),float(row[3]),float(row[4]),float(row[5]),float(row[6]),float(row[7]),int(row[8])])
finalDataRuns = data.map(lambda seq: LabeledPoint(seq[-1],seq[0:13]))
finalDataRuns.take(2)

[LabeledPoint(0.0, [26.41958042,131.63763070000002,0.043554007,0.131358885,33.1125,7.8026509572900995,25.4625,0.79207920792079,0.0]),
 LabeledPoint(2.0, [26.41958042,131.63763070000002,0.043554007,0.131358885,33.1125,7.8026509572900995,25.4625,0.79207920792079,2.0])]

In [5]:
data = spark.read.format("csv").load("no_char_join_runs.csv")
data = data.rdd
#finalData = data.map(lambda row: [float(row[0]),float(row[1]),float(row[2]),float(row[3]),float(row[4]),float(row[5]),float(row[6]),float(row[7]),int(row[8])])
finalDataRuns = data.map(lambda seq: LabeledPoint(seq[-1],seq[0:13]))
finalDataRuns.take(2)

[LabeledPoint(11.0, [0.0,26.41958042,131.63763070000002,0.043554007,0.131358885,33.1125,7.8026509572900995,25.4625,0.79207920792079,25.046875,130.4312449,0.048006509,0.138323841]),
 LabeledPoint(0.0, [0.0,26.41958042,131.63763070000002,0.043554007,0.131358885,33.1125,7.8026509572900995,25.4625,0.79207920792079,25.046875,130.4312449,0.048006509,0.138323841])]

In [6]:
data = spark.read.format("csv").load("no_char_join_wickets.csv")
data = data.rdd
#finalData = data.map(lambda row: [float(row[0]),float(row[1]),float(row[2]),float(row[3]),float(row[4]),float(row[5]),float(row[6]),float(row[7]),int(row[8])])
finalDataWickets = data.map(lambda seq: LabeledPoint(seq[-1],seq[0:13]))
finalDataWickets.take(2)

[LabeledPoint(0.0, [0.0,26.41958042,131.63763070000002,0.043554007,0.131358885,33.1125,7.8026509572900995,25.4625,0.79207920792079,25.046875,130.4312449,0.048006509,0.138323841]),
 LabeledPoint(0.0, [0.0,26.41958042,131.63763070000002,0.043554007,0.131358885,33.1125,7.8026509572900995,25.4625,0.79207920792079,25.046875,130.4312449,0.048006509,0.138323841])]

In [7]:
(trainingDataW, testDataW) = finalDataWickets.randomSplit([0.7, 0.3])
(trainingDataR, testDataR) = finalDataRuns.randomSplit([0.7, 0.3])

In [8]:
# Ball by ball
modelR = DecisionTree.trainRegressor(trainingDataR, categoricalFeaturesInfo={},impurity='variance', maxDepth=5, maxBins=32)

In [862]:
# over by over
# 16 and 35 gives 5/9
# 30 and 50 average
# 10 and 20 gives 5/8 pretty good
# 10 and 25 gives 5/8 pretty good
# 22 and 38 more right results but fewer closer scores 6/8
# 17 and 38 not that great
# 8 and 20 gives 3 2 3
# 11 and 32 gives 3 3 2
# 14 and 32 is better than 11 and 32
final_i =0
final_j=0
minE=100
for i in range(5,31):
    for j in range(5,31):
        modelR = DecisionTree.trainRegressor(trainingDataR, categoricalFeaturesInfo={},impurity='variance', maxDepth=i, maxBins=32)
        modelW = DecisionTree.trainRegressor(trainingDataW,categoricalFeaturesInfo={},impurity='variance', maxDepth=j, maxBins=32)
        predictionsR = modelR.predict(testDataR.map(lambda x: x.features))
        predictionsR = predictionsR.collect()
        predictionsR = [round(x) for x in predictionsR]
#         print("Runs: ",sum(predictionsR))

        predictionsW = modelW.predict(testDataW.map(lambda x: x.features))
        predictionsW = predictionsW.collect()
        predictionsW = [round(x) for x in predictionsW]
#         print("Wickets: ",sum(predictionsW))
        c=0.0
        for i1 in range(len(runs)):
            c+=(runs[i1]-predictionsR[i1])**2
        testerr1 = c/len(runs)
        print("Error in Runs: ",testerr1)
        c=0.0
        for i1 in range(len(wickets)):
            c+=(wickets[i1]-predictionsW[i1])**2
        testerr2 = c/len(wickets)
        print("Error in Wickets: ",testerr2)
        if(testerr1+testerr2<minE):
            minE=testerr1+testerr2
            final_i=i
            final_j=j

Error in Runs:  19.878770871334364
Error in Wickets:  0.3749647589512264
Error in Runs:  19.878770871334364
Error in Wickets:  0.37665632929235976
Error in Runs:  19.878770871334364
Error in Wickets:  0.38342261065689315
Error in Runs:  19.878770871334364
Error in Wickets:  0.40062024245841554
Error in Runs:  19.878770871334364
Error in Wickets:  0.4041443473357767
Error in Runs:  19.878770871334364
Error in Wickets:  0.422892585283338
Error in Runs:  19.878770871334364
Error in Wickets:  0.4430504651818438
Error in Runs:  19.878770871334364
Error in Wickets:  0.45672399210600506
Error in Runs:  19.878770871334364
Error in Wickets:  0.47039751903016636
Error in Runs:  19.878770871334364
Error in Wickets:  0.482943332393572
Error in Runs:  19.878770871334364
Error in Wickets:  0.50380603326755
Error in Runs:  19.878770871334364
Error in Wickets:  0.5174795601917113
Error in Runs:  19.878770871334364
Error in Wickets:  0.5331265858471949
Error in Runs:  19.878770871334364
Error in Wicket

Error in Runs:  22.05121369440157
Error in Wickets:  0.482943332393572
Error in Runs:  22.05121369440157
Error in Wickets:  0.50380603326755
Error in Runs:  22.05121369440157
Error in Wickets:  0.5174795601917113
Error in Runs:  22.05121369440157
Error in Wickets:  0.5331265858471949
Error in Runs:  22.05121369440157
Error in Wickets:  0.5499013250634339
Error in Runs:  22.05121369440157
Error in Wickets:  0.5645616013532563
Error in Runs:  22.05121369440157
Error in Wickets:  0.5749929517902452
Error in Runs:  22.05121369440157
Error in Wickets:  0.5816182689596843
Error in Runs:  22.05121369440157
Error in Wickets:  0.5899351564702565
Error in Runs:  22.05121369440157
Error in Wickets:  0.5947279391034678
Error in Runs:  22.05121369440157
Error in Wickets:  0.59923879334649
Error in Runs:  22.05121369440157
Error in Wickets:  0.6024809698336623
Error in Runs:  22.05121369440157
Error in Wickets:  0.604595432760079
Error in Runs:  22.05121369440157
Error in Wickets:  0.604454468564984

Error in Runs:  27.061737056264906
Error in Wickets:  0.59923879334649
Error in Runs:  27.061737056264906
Error in Wickets:  0.6024809698336623
Error in Runs:  27.061737056264906
Error in Wickets:  0.604595432760079
Error in Runs:  27.061737056264906
Error in Wickets:  0.6044544685649845
Error in Runs:  27.061737056264906
Error in Wickets:  0.6058641105159289
Error in Runs:  27.061737056264906
Error in Wickets:  0.6058641105159289
Error in Runs:  27.061737056264906
Error in Wickets:  0.6067098956864956
Error in Runs:  29.130489687105374
Error in Wickets:  0.3749647589512264
Error in Runs:  29.130489687105374
Error in Wickets:  0.37665632929235976
Error in Runs:  29.130489687105374
Error in Wickets:  0.38342261065689315
Error in Runs:  29.130489687105374
Error in Wickets:  0.40062024245841554
Error in Runs:  29.130489687105374
Error in Wickets:  0.4041443473357767
Error in Runs:  29.130489687105374
Error in Wickets:  0.422892585283338
Error in Runs:  29.130489687105374
Error in Wickets:

Error in Runs:  35.38501473270661
Error in Wickets:  0.40062024245841554
Error in Runs:  35.38501473270661
Error in Wickets:  0.4041443473357767
Error in Runs:  35.38501473270661
Error in Wickets:  0.422892585283338
Error in Runs:  35.38501473270661
Error in Wickets:  0.4430504651818438
Error in Runs:  35.38501473270661
Error in Wickets:  0.45672399210600506
Error in Runs:  35.38501473270661
Error in Wickets:  0.47039751903016636
Error in Runs:  35.38501473270661
Error in Wickets:  0.482943332393572
Error in Runs:  35.38501473270661
Error in Wickets:  0.50380603326755
Error in Runs:  35.38501473270661
Error in Wickets:  0.5174795601917113
Error in Runs:  35.38501473270661
Error in Wickets:  0.5331265858471949
Error in Runs:  35.38501473270661
Error in Wickets:  0.5499013250634339
Error in Runs:  35.38501473270661
Error in Wickets:  0.5645616013532563
Error in Runs:  35.38501473270661
Error in Wickets:  0.5749929517902452
Error in Runs:  35.38501473270661
Error in Wickets:  0.5816182689

Error in Runs:  38.497965483373086
Error in Wickets:  0.5645616013532563
Error in Runs:  38.497965483373086
Error in Wickets:  0.5749929517902452
Error in Runs:  38.497965483373086
Error in Wickets:  0.5816182689596843
Error in Runs:  38.497965483373086
Error in Wickets:  0.5899351564702565
Error in Runs:  38.497965483373086
Error in Wickets:  0.5947279391034678
Error in Runs:  38.497965483373086
Error in Wickets:  0.59923879334649
Error in Runs:  38.497965483373086
Error in Wickets:  0.6024809698336623
Error in Runs:  38.497965483373086
Error in Wickets:  0.604595432760079
Error in Runs:  38.497965483373086
Error in Wickets:  0.6044544685649845
Error in Runs:  38.497965483373086
Error in Wickets:  0.6058641105159289
Error in Runs:  38.497965483373086
Error in Wickets:  0.6058641105159289
Error in Runs:  38.497965483373086
Error in Wickets:  0.6067098956864956
Error in Runs:  38.96814929142697
Error in Wickets:  0.3749647589512264
Error in Runs:  38.96814929142697
Error in Wickets:  0.

Error in Runs:  39.5119966325242
Error in Wickets:  0.6067098956864956
Error in Runs:  39.56180721201066
Error in Wickets:  0.3749647589512264
Error in Runs:  39.56180721201066
Error in Wickets:  0.37665632929235976
Error in Runs:  39.56180721201066
Error in Wickets:  0.38342261065689315
Error in Runs:  39.56180721201066
Error in Wickets:  0.40062024245841554
Error in Runs:  39.56180721201066
Error in Wickets:  0.4041443473357767
Error in Runs:  39.56180721201066
Error in Wickets:  0.422892585283338
Error in Runs:  39.56180721201066
Error in Wickets:  0.4430504651818438
Error in Runs:  39.56180721201066
Error in Wickets:  0.45672399210600506
Error in Runs:  39.56180721201066
Error in Wickets:  0.47039751903016636
Error in Runs:  39.56180721201066
Error in Wickets:  0.482943332393572
Error in Runs:  39.56180721201066
Error in Wickets:  0.50380603326755
Error in Runs:  39.56180721201066
Error in Wickets:  0.5174795601917113
Error in Runs:  39.56180721201066
Error in Wickets:  0.533126585

KeyboardInterrupt: 

In [ ]:
final_i =16
final_j=44
minE=19.77496722576557
for i in range(16,50):
    ctr=0
    modelR = DecisionTree.trainRegressor(trainingDataR, categoricalFeaturesInfo={},impurity='variance', maxDepth=5, maxBins=i)
    for j in range(5,50):
        modelW = DecisionTree.trainRegressor(trainingDataW,categoricalFeaturesInfo={},impurity='variance', maxDepth=5, maxBins=j)
        predictionsR = modelR.predict(testDataR.map(lambda x: x.features))
        predictionsR = predictionsR.collect()
        predictionsR = [round(x) for x in predictionsR]
#         print("Runs: ",sum(predictionsR))

        predictionsW = modelW.predict(testDataW.map(lambda x: x.features))
        predictionsW = predictionsW.collect()
        predictionsW = [round(x) for x in predictionsW]
#         print("Wickets: ",sum(predictionsW))
        runs = testDataR.map(lambda x: x.label)
        runs = runs.collect()
        #print("Runs: ",sum(runs))

        wickets = testDataW.map(lambda x: x.label)
        wickets = wickets.collect()
        #print("Wickets: ",sum(wickets))
        ctr=ctr+1
        if(ctr==1):
            c=0.0
            for i1 in range(len(runs)):
                c+=(runs[i1]-predictionsR[i1])**2
            testerr1 = c/len(runs)
            print("Error in Runs: ",testerr1)
        c=0.0
        for i1 in range(len(wickets)):
            c+=(wickets[i1]-predictionsW[i1])**2
        testerr2 = c/len(wickets)
        print("Error in Wickets: ",testerr2,"  ",j)
        if(testerr1+testerr2<minE):
            minE=testerr1+testerr2
            final_i=i
            final_j=j

In [9]:
modelR = DecisionTree.trainRegressor(trainingDataR, categoricalFeaturesInfo={},impurity='variance', maxDepth=5, maxBins=32)
modelW = DecisionTree.trainRegressor(trainingDataW,categoricalFeaturesInfo={},impurity='variance', maxDepth=22, maxBins=52)

In [10]:
predictionsR = modelR.predict(testDataR.map(lambda x: x.features))
predictionsR = predictionsR.collect()
predictionsR = [round(x) for x in predictionsR]
print("Runs: ",sum(predictionsR))

predictionsW = modelW.predict(testDataW.map(lambda x: x.features))
predictionsW = predictionsW.collect()
predictionsW = [round(x) for x in predictionsW]
print("Wickets: ",sum(predictionsW))

Runs:  56404
Wickets:  2266


In [ ]:
runs = testDataR.map(lambda x: x.label)
runs = runs.collect()
print("Runs: ",sum(runs))

wickets = testDataW.map(lambda x: x.label)
wickets = wickets.collect()
print("Wickets: ",sum(wickets))

In [ ]:
c=0.0
for i in range(len(runs)):
    c+=(runs[i]-predictionsR[i])**2
testerr = c/len(runs)
print("Error in Runs: ",testerr)
c=0.0
for i in range(len(wickets)):
    c+=(wickets[i]-predictionsW[i])**2
testerr = c/len(wickets)
print("Error in Wickets: ",testerr)

In [ ]:
batting_stats = pd.read_csv("Batting_Cluster_Final.csv")
bowling_stats = pd.read_csv("Bowling_Cluster_Final.csv")
batting_stats.columns=["Player Name","Average","Strike Rate","Bleh","Sixes","Fours"]
bowling_stats.columns = ["Player Name","Average","Economy","Strike Rate","WicketsProb"]
batting_stats = batting_stats.drop("Bleh",axis=1)

In [16]:
bat_cluster = pd.read_csv("batting_cluster.csv")
bowl_cluster = pd.read_csv("bowling_cluster.csv")
bat_cluster.columns = ["Name","Cluster"]
bowl_cluster.columns = ["Name","Cluster"]
# x = bat_cluster[bat_cluster["Name"]=="CR Woakes"]
# print(x)
def call_cluster(name,kind):
    count = 0
    total = {'Average':0,'Strike Rate':0,'Sixes':0,'Fours':0}
    total1 = {'Average':0,'Economy':0,'Strike Rate':0,'WicketsProb':0}
    if(kind=="Batsman"):
        c_no = int(bat_cluster[bat_cluster["Name"]==name]["Cluster"])
        batsmen = bat_cluster[bat_cluster["Cluster"]==c_no]["Name"]
        batsmen = list(batsmen)
        for i in batsmen:
            bat = batting_stats[batting_stats["Player Name"]==i]
            if(sys.getsizeof(bat)>24):
                count+=1
                total['Average']=total['Average']+float(bat['Average'])
                total["Strike Rate"]+=float(bat["Strike Rate"])
                total["Sixes"]+=float(bat["Sixes"])
                total["Fours"]+=float(bat["Fours"])
        total["Average"]/=count
        total["Strike Rate"]/=count
        total["Sixes"]/=count
        total["Fours"]/=count
        return total
    else:
        c_no = int(bowl_cluster[bowl_cluster["Name"]==name]["Cluster"])
        bowlers = bowl_cluster[bowl_cluster["Cluster"]==c_no]["Name"]
        bowlers = list(bowlers)
        for i in bowlers:
            bowl = bowling_stats[bowling_stats["Player Name"]==i]
            if(sys.getsizeof(bowl)>24):
                count+=1
                total1['Average']=total1['Average']+float(bowl['Average'])
                total1["Strike Rate"]+=float(bowl["Strike Rate"])
                total1["Economy"]+=float(bowl["Economy"])
                total1["WicketsProb"]+=float(bowl["WicketsProb"])
        total1["Average"]/=count
        total1["Strike Rate"]/=count
        total1["Economy"]/=count
        total1["WicketsProb"]/=count
        return total1

In [17]:
# Code to test the function
import sys
bowl = bowling_stats[bowling_stats["Player Name"]=="xi"]
if(sys.getsizeof(bowl)==24):
    stats = call_cluster("A Kumble","Bowler")
    print(stats)

{'Average': 27.99069108779549, 'Economy': 8.510502842938315, 'Strike Rate': 20.549508564043464, 'WicketsProb': 0.7132786799005538}


In [18]:
#1.CSK vs MI 28 apr
# team1_batting_order=["SR Watson","AT Rayudu","SK Raina","SW Billings","RA Jadeja","MS Dhoni","DJ Bravo","Harbhajan Singh","DL Chahar","SN Thakur","Imran Tahir"]
# team2_batting_order=["AS Yadav","Evin Lewis","Ishan Kishan","RG Sharma","JP Duminy","KH Pandya","HH Pandya","BCJ Cutting","MJ McClenaghan","SL Malinga","JJ Bumrah"]
# team1_bowling_order=["DL Chahar","SN Thakur","DL Chahar","SR Watson","DL Chahar","SN Thakur","SR Watson","Harbhajan Singh","Imran Tahir","Harbhajan Singh","Imran Tahir","Harbhajan Singh","DJ Bravo","SR Watson","DJ Bravo","SN Thakur","DJ Bravo","SR Watson","SN Thakur","Imran Tahir"]
# team2_bowling_order=["MJ McClenaghan","JJ Bumrah","MJ McClenaghan","HH Pandya","KH Pandya","HH Pandya","KH Pandya","SL Malinga","BCJ Cutting","SL Malinga","MJ McClenaghan","KH Pandya","JJ Bumrah","KH Pandya","HH Pandya","SL Malinga","JJ Bumrah","MJ McClenaghan","JJ Bumrah","HH Pandya"]

# MI vs DD 9
# team1_batting_order=["AS Yadav","Evin Lewis","Ishan Kishan","RG Sharma","KA Pollard","KH Pandya","HH Pandya","A Dananjaya","Mayank Markande","JJ Bumrah","Mustafizur Rahman"]
# team2_batting_order=["JJ Roy","G Gambhir","RR Pant","GJ Maxwell","SS Iyer","V Shankar","DT Christian","R Tewatia","S Nadeem","Mohammed Shami","TA Boult"]
# team1_bowling_order=["HH Pandya","A Dananjaya","JJ Bumrah","Mustafizur Rahman","HH Pandya","Mustafizur Rahman","Mayank Markande","A Dananjaya","Mayank Markande","A Dananjaya","JJ Bumrah","KH Pandya","Mayank Markande","KH Pandya","A Dananjaya","KH Pandya","JJ Bumrah","Mustafizur Rahman","JJ Bumrah","Mustafizur Rahman"]
# team2_bowling_order=["TA Boult","S Nadeem","TA Boult","S Nadeem","Mohammed Shami","DT Christian","R Tewatia","GJ Maxwell","R Tewatia","GJ Maxwell","R Tewatia","GJ Maxwell","R Tewatia","DT Christian","Mohammed Shami","DT Christian","Mohammed Shami","TA Boult","Mohammed Shami","TA Boult"]

#RR vs KKR 49
# team1_batting_order=["RA Tripathi","JC Buttler","AM Rahane","SV Samson","BA Stokes","STR Binny","Krishnappa Gowtham","JD Unadkat","Ish Sodhi","Jofra Archer","A Singh"]
# team2_batting_order=["SP Narine","CA Lynn","RV Uthappa","S Rana","KD Karthik","AD Russell","S Gill","Shivam Mavi","Javon Searles","Prasidh Krishna","Kuldeep Yadav"]
# team1_bowling_order=["Krishnappa Gowtham","BA Stokes","Jofra Archer","BA Stokes","Jofra Archer","BA Stokes","Ish Sodhi","Krishnappa Gowtham","Ish Sodhi","JD Unadkat","Ish Sodhi","JD Unadkat","A Singh","Jofra Archer","Ish Sodhi","BA Stokes","JD Unadkat","Jofra Archer","Krishnappa Gowtham","A Singh"]
# team2_bowling_order=["Shivam Mavi","Prasidh Krishna","Shivam Mavi","SP Narine","AD Russell","SP Narine","AD Russell","Kuldeep Yadav","Prasidh Krishna","Kuldeep Yadav","SP Narine","Kuldeep Yadav","Shivam Mavi","Kuldeep Yadav","Shivam Mavi","SP Narine","Prasidh Krishna","AD Russell","Prasidh Krishna","AD Russell"]

#DD vs RCB 19
# team2_batting_order=["Q de Kock","M Vohra","V Kohli","AB de Villiers","CJ Anderson","Mandeep Singh","Washington Sundar","CR Woakes","UT Yadav","Mohammed Siraj","YS Chahal"]
# team1_batting_order=["JJ Roy","G Gambhir","SS Iyer","RR Pant","GJ Maxwell","R Tewatia","CH Morris","V Shankar","S Nadeem","HV Patel","TA Boult"]
# team1_bowling_order=["TA Boult","GJ Maxwell","TA Boult","GJ Maxwell","S Nadeem","CH Morris","S Nadeem","CH Morris","HV Patel","R Tewatia","HV Patel","R Tewatia","HV Patel","R Tewatia","CH Morris","TA Boult","CH Morris","TA Boult","HV Patel","GJ Maxwell"]
# team2_bowling_order=["UT Yadav","CR Woakes","UT Yadav","YS Chahal","UT Yadav","YS Chahal","Mohammed Siraj","Washington Sundar","UT Yadav","Washington Sundar","CR Woakes","Washington Sundar","Mohammed Siraj","Washington Sundar","Mohammed Siraj","YS Chahal","CR Woakes","Mohammed Siraj","CR Woakes","CJ Anderson"]

#RR vs RCB 11
# team1_batting_order=["AM Rahane","D'Arcy Short","SV Samson","BA Stokes","JC Buttler","RA Tripathi","Krishnappa Gowtham","S Gopal","DS Kulkarni","JD Unadkat","B Laughlin"]
# team2_batting_order=["BB McCullum","Q de Kock","V Kohli","AB de Villiers","Mandeep Singh","P Negi","Washington Sundar","CR Woakes","Kulwant Khejroliya","UT Yadav","YS Chahal"]
# team2_bowling_order=["Washington Sundar","CR Woakes","Washington Sundar","UT Yadav","YS Chahal","CR Woakes","YS Chahal","Kulwant Khejroliya","Washington Sundar","UT Yadav","Washington Sundar","Kulwant Khejroliya","YS Chahal","P Negi","YS Chahal","UT Yadav","CR Woakes","Kulwant Khejroliya","CR Woakes","UT Yadav"]
# team1_bowling_order=["Krishnappa Gowtham","DS Kulkarni","Krishnappa Gowtham","JD Unadkat","Krishnappa Gowtham","BA Stokes","S Gopal","D Short","S Gopal","B Laughlin","S Gopal","JD Unadkat","S Gopal","Krishnappa Gowtham","B Laughlin","BA Stokes","JD Unadkat","B Laughlin","BA Stokes","B Laughlin"]                     

#CSK vs KXIP 12
# team2_batting_order = ["SR Watson", "M Vijay", "AT Rayudu", "SW Billings", "MS Dhoni","RA Jadeja", "DJ Bravo", "DL Chahar", "Harbhajan Singh", "Imran Tahir", "SN Thakur"]
# team1_batting_order = ["KL Rahul", "CH Gayle", "MA Agarwal", "Yuvraj Singh", "AJ Finch", "KK Nair", "R Ashwin", "AJ Tye", "BB Sran", "MM Sharma", "Mujeeb Ur Rahman"]
# team2_bowling_order = ["DL Chahar", "Harbhajan Singh", "DL Chahar", "Harbhajan Singh", "SN Thakur", "DL Chahar", "Imran Tahir", "Harbhajan Singh", "SR Watson", "Harbhajan Singh", "DJ Bravo","SR Watson", "Imran Tahir", "DJ Bravo", "Imran Tahir",  "SN Thakur", "Imran Tahir", "DJ Bravo", "SN Thakur", "DJ Bravo"]
# team1_bowling_order = ["BB Sran", "MM Sharma", "BB Sran", "MM Sharma", "AJ Tye", "Mujeeb Ur Rahman", "R Ashwin", "Yuvraj Singh", "R Ashwin", "Mujeeb Ur Rahman", "AJ Tye", "Mujeeb Ur Rahman","R Ashwin", "BB Sran", "R Ashwin", "BB Sran", "AJ Tye", "MM Sharma", "AJ Tye", "MM Sharma"]

#SRH vs CSK Final
team1_batting_order=["SP Goswami","S Dhawan","KS Williamson","Shakib Al Hasan","YK Pathan","DJ Hooda","CR Brathwaite","Rashid Khan","B Kumar","S Kaul","Sandeep Sharma"]
team2_batting_order=["SR Watson","F du Plessis","SK Raina","AT Rayudu","MS Dhoni","DJ Bravo","RA Jadeja","KV Sharma","DL Chahar","SN Thakur","Imran Tahir"]#lungi at 11
team1_bowling_order=["B Kumar","Sandeep Sharma","B Kumar","Sandeep Sharma","B Kumar","Sandeep Sharma","S Kaul","Rashid Khan","S Kaul","Rashid Khan","Shakib Al Hasan","CR Brathwaite","Sandeep Sharma","CR Brathwaite","Rashid Khan","B Kumar","Rashid Khan","S Kaul","CR Brathwaite","S Kaul"]
team2_bowling_order=["DL Chahar","L Ngidi","DL Chahar","L Ngidi","DL Chahar","SN Thakur","KV Sharma","DJ Bravo","RA Jadeja","DL Chahar","RA Jadeja","DJ Bravo","KV Sharma","SN Thakur","KV Sharma","DJ Bravo","L Ngidi","DJ Bravo","L Ngidi","SN Thakur"]

#SRH vs CSK QUALIFIER 1
# team2_bowling_order=["DL Chahar","L Ngidi","DL Chahar","L Ngidi","SN Thakur","DL Chahar","DJ Bravo","RA Jadeja","DJ Bravo","RA Jadeja","DL Chahar","RA Jadeja","SN Thakur","RA Jadeja","DJ Bravo","L Ngidi","DJ Bravo","SN Thakur","L Ngidi","SN Thakur"]
# team1_bowling_order=["B Kumar","Sandeep Sharma","B Kumar","S Kaul","B Kumar","S Kaul","CR Brathwaite","Rashid Khan","CR Brathwaite","Rashid Khan","Shakib Al Hasan","Rashid Khan","Sandeep Sharma","Shakib Al Hasan","Sandeep Sharma","Rashid Khan","S Kaul","CR Brathwaite","S Kaul","B Kumar"]
# team1_batting_order=["S Dhawan","SP Goswami","KS Williamson","MK Pandey","Shakib Al Hasan","YK Pathan","CR Brathwaite","B Kumar","Rashid Khan","S Kaul","Sandeep Sharma"]
# team2_batting_order=["SR Watson","F du Plessis","SK Raina","AT Rayudu","MS Dhoni","DJ Bravo","RA Jadeja","DL Chahar","Harbhajan Singh","SN Thakur"]

#CSK vs MI 27
# team1_batting_order=["SR Watson","AT Rayudu","SK Raina","SW Billings","RA Jadeja","MS Dhoni","DJ Bravo","Harbhajan Singh","DL Chahar","SN Thakur","Imran Tahir"]
# team2_batting_order=["AS Yadav","Evin Lewis","Ishan Kishan","RG Sharma","JP Duminy","KH Pandya","HH Pandya","BCJ Cutting","MJ McClenaghan","SL Malinga","JJ Bumrah"]
# team1_bowling_order=["DL Chahar","SN Thakur","DL Chahar","SR Watson","DL Chahar","SN Thakur","SR Watson","Harbhajan Singh","Imran Tahir","Harbhajan Singh","Imran Tahir","Harbhajan Singh","DJ Bravo","SR Watson","DJ Bravo","SN Thakur","DJ Bravo","SR Watson","SN Thakur","Imran Tahir"]
# team2_bowling_order=["MJ McClenaghan","JJ Bumrah","MJ McClenaghan","HH Pandya","KH Pandya","HH Pandya","KH Pandya","SL Malinga","BCJ Cutting","SL Malinga","MJ McClenaghan","KH Pandya","JJ Bumrah","KH Pandya","HH Pandya","SL Malinga","JJ Bumrah","MJ McClenaghan","JJ Bumrah","HH Pandya"]


# team1_batting_order = ["C Munro","G Gambhir", "SS Iyer","V Shankar", "RR Pant", "R Tewatia", "CH Morris", "DT Christian", "A Mishra", "TA Boult", "Mohammed Shami"]
# team2_batting_order = ["KL Rahul", "MA Agarwal","Yuvraj Singh", "KK Nair", "DA Miller", "MP Stoinis", "AR Patel", "R Ashwin", "AJ Tye", "MM Sharma", "Mujeeb Ur Rahman"]
# team1_bowling_order = ["TA Boult", "Mohammed Shami", "A Mishra", "CH Morris", "TA Boult", "CH Morris", "TA Boult", "DT Christian", "Mohammed Shami","R Tewatia", "A Mishra", "R Tewatia", "A Mishra", "R Tewatia", "A Mishra", "DT Christian", "R Tewatia", "CH Morris", "TA Boult", "DT Christian"]
# team2_bowling_order = ["R Ashwin", "MM Sharma", "Mujeeb Ur Rahman", "AR Patel", "Mujeeb Ur Rahman", "AJ Tye", "AR Patel", "R Ashwin", "AR Patel", "MP Stoinis","MM Sharma", "R Ashwin", "AJ Tye", "Mujeeb Ur Rahman", "AJ Tye", "R Ashwin", "MM Sharma", "Mujeeb Ur Rahman", "AJ Tye", "MM Sharma"]




In [19]:
from math import trunc
def simulate_innings(batting_order,bowling_order,target):
    overnum=0
    list_pass=[overnum]
    batsman = batting_order[0]
    nonstriker = batting_order[1]
    bowler = bowling_order[overnum]
    runs = 0
    victoryFlag = 0
    wickets = 0
    while(overnum<20 and wickets<10 and victoryFlag==0):
        list_pass=[overnum]
        bowler = bowling_order[overnum]
#         print(bowler,batsman,nonstriker)
        res=batting_stats[batting_stats["Player Name"]==batsman]
        if(sys.getsizeof(res)<=24):
            res = call_cluster(batsman,"Batsman")
        list_pass.append(float(res["Average"]))
        list_pass.append(float(res["Strike Rate"]))
        list_pass.append(float(res["Sixes"]))
        list_pass.append(float(res["Fours"]))
        res=bowling_stats[bowling_stats["Player Name"]==bowler]
        if(sys.getsizeof(res)<=24):
            res = call_cluster(bowler,"Bowler")
        list_pass.append(float(res["Average"]))
        list_pass.append(float(res["Economy"]))
        list_pass.append(float(res["Strike Rate"]))
        list_pass.append(float(res["WicketsProb"]))
        res=batting_stats[batting_stats["Player Name"]==nonstriker]
        if(sys.getsizeof(res)<=24):
            res = call_cluster(nonstriker,"Batsman")
        list_pass.append(float(res["Average"]))
        list_pass.append(float(res["Strike Rate"]))
        list_pass.append(float(res["Sixes"]))
        list_pass.append(float(res["Fours"]))
#         print(list_pass)
        overnum+=1;
        overruns = modelR.predict(list_pass)
        overruns = round(overruns)
        if(overruns>36):
            overruns=36
        overwickets = modelW.predict(list_pass)
        overwickets = round(overwickets)
        if(overwickets>6):
            overwickets = 6
        runs_per_ball = overruns/6
        overruns = overruns - (round(runs_per_ball*overwickets))
        runs += overruns
        wickets += overwickets
        if(wickets>10):
            wickets=10
        print("Runs in over ",overnum,": ",overruns," and total runs is: ",runs)
        print("Wickets in over ",overnum,": ",overwickets," and total wickets is: ",wickets)
        if(wickets >= 10):
            break
        elif(overwickets!=0):
            batsman = batting_order[wickets]
            nonstriker = batting_order[wickets+1]
        if(overruns%2==0):
            temp = batsman
            batsman = nonstriker
            nonstriker = temp
        if(target!=0 and runs > target):
            victoryFlag=1            
    return (runs,wickets)


In [20]:
team1 = simulate_innings(team1_batting_order, team2_bowling_order, 0)
print("Team 1 total: ",team1[0]," and wickets: ",team1[1])
print()
team2 = simulate_innings(team2_batting_order, team1_bowling_order, team1[0])
print("Team 2 total: ",team2[0]," and wickets: ",team2[1])
if(team1[0]>team2[0]):
    print("TEAM 1 WINS")
else:
    print("TEAM 2 WINS")

Runs in over  1 :  5  and total runs is:  5
Wickets in over  1 :  0  and total wickets is:  0
Runs in over  2 :  7  and total runs is:  12
Wickets in over  2 :  0  and total wickets is:  0
Runs in over  3 :  5  and total runs is:  17
Wickets in over  3 :  0  and total wickets is:  0
Runs in over  4 :  6  and total runs is:  23
Wickets in over  4 :  1  and total wickets is:  1
Runs in over  5 :  7  and total runs is:  30
Wickets in over  5 :  1  and total wickets is:  2
Runs in over  6 :  9  and total runs is:  39
Wickets in over  6 :  0  and total wickets is:  2
Runs in over  7 :  7  and total runs is:  46
Wickets in over  7 :  0  and total wickets is:  2
Runs in over  8 :  6  and total runs is:  52
Wickets in over  8 :  1  and total wickets is:  3
Runs in over  9 :  8  and total runs is:  60
Wickets in over  9 :  0  and total wickets is:  3
Runs in over  10 :  12  and total runs is:  72
Wickets in over  10 :  0  and total wickets is:  3
Runs in over  11 :  8  and total runs is:  80
Wi

In [21]:
# print(modelR.toDebugString())

In [22]:
# print(modelW.toDebugString())